In [1]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go
import mplfinance as mpf
import time
import settrade_v2
from settrade_v2 import Investor

In [2]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

In [3]:
keyspace_name = 'stock_data'

session.execute(f"CREATE KEYSPACE IF NOT EXISTS {keyspace_name} WITH REPLICATION = {{ 'class': 'SimpleStrategy', 'replication_factor': 1 }};")
session.set_keyspace(keyspace_name)

In [4]:
# ดึง keyspace ทั้งหมด
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")

# แสดงผลลัพธ์
print("Available Keyspaces:")
for row in rows:
    print("-", row.keyspace_name)

Available Keyspaces:
- system_auth
- system_schema
- system_distributed
- system
- system_traces
- stock_data


In [8]:
# สร้างตาราง stock_price หากยังไม่มี
insert_query = session.prepare("""
    INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
    VALUES (?, ?, ?, ?, ?, ?, ?)
""")

In [9]:

investor = Investor(
    app_id="ZGCsmM0t5c9VMLRJ",                                 
    app_secret="T5eQ70WLmOWDpVvV7jicJtspuBR2A361nzVGmR8xTF4=", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)

deri = investor.Derivatives(account_no="Hilman-D")
account_info = deri.get_account_info()

account_info 

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [13]:
# Get real-time quote
market_data = investor.MarketData()
stock = market_data.get_quote_symbol("AOT")
print(stock)

SettradeError: User is inactive

In [ ]:
# get candlestick
market_cd = investor.MarketData()
res = market_cd.get_candlestick(symbol="AOT", interval="1d",normalized=True,)

SettradeError: User is inactive

In [ ]:
# 👀 ตรวจสอบก่อนว่า res เป็น dict และมี key 'data' หรือไม่
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,156215,1683824400,73.50,73.75,73.00,73.25,23390128,0.0
1,156215,1684083600,73.75,73.75,73.00,73.25,37639520,0.0
2,156215,1684170000,73.25,73.50,72.00,72.00,32810851,0.0
3,156215,1684256400,72.00,72.25,71.00,72.00,48477889,0.0
4,156215,1684342800,72.00,72.50,71.50,71.50,25320243,0.0
...,...,...,...,...,...,...,...,...
495,156215,1747933200,34.50,34.50,33.75,34.00,12024419,0.0
496,156215,1748192400,34.00,34.50,33.75,34.00,14216637,0.0
497,156215,1748278800,34.00,34.00,33.00,33.00,19693613,0.0
498,156215,1748365200,33.25,34.00,33.00,33.75,18291272,0.0


In [ ]:
df.columns

Index(['lastSequence', 'time', 'open', 'high', 'low', 'close', 'volume',
       'value'],
      dtype='object')

In [ ]:
# แปลงวันที่ให้เป็น datetime object
df["time"] = pd.to_datetime(df["time"])
df.set_index("time", inplace=True)

# วาดกราฟ
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["open"],
    high=df["high"],
    low=df["low"],
    close=df["close"],
    name="Candlestick"
))

fig.add_trace(go.Bar(
    x=df.index,
    y=df["volume"],
    name="Volume",
    yaxis='y2',
    marker_color='rgba(150,150,150,0.3)'
))

fig.update_layout(
    title="กราฟแท่งเทียนหุ้น (จาก Settrade API)",
    xaxis_rangeslider_visible=False,
    yaxis=dict(title="ราคา"),
    yaxis2=dict(title="Volume", overlaying="y", side="right", showgrid=False),
    height=600,
)

fig.show()

In [ ]:
session.set_keyspace('stock_data')

In [ ]:
#GetQuoteSymbol
mkt_data = investor.MarketData()
res = mkt_data.get_quote_symbol("AOT")
print(res)

{'instrumentType': 'STOCK', 'symbol': 'AOT', 'high': 58.75, 'low': 41.75, 'last': 43.25, 'average': 52.76154, 'change': -15.25, 'percentChange': -26.07, 'totalVolume': 2843300, 'totalBuyVolume': 680800, 'totalSellVolume': 2110600, 'totalNoSideVolume': 51900, 'status': '', 'marketStatus': 'Open2', 'securityType': 'CS', 'eps': 0.72783, 'pe': 29.53, 'pbv': 4.61, 'percentYield': 1.9900000095367432, 'maturityDate': None, 'exercisePrice': None, 'underlying': None, 'underlyingPrice': None, 'intrinsicValue': None, 'theoretical': None, 'moneyness': None, 'lastTradingDate': None, 'toLastTrade': None, 'exerciseRatio': None, 'impliedVolatility': None, 'exchange': None, 'aumSize': None, 'inav': None}


In [ ]:

def to_date(val):
    try:
        return datetime.strptime(val, "%Y-%m-%d").date() if val else None
    except Exception:
        return None

try:
    session.execute(insert_query, (
        res.get("symbol"),
        datetime.now().date(),  # ใช้วันที่ปัจจุบันหรือกำหนดเอง
        res.get("open") if res.get("open") is not None else res.get("last"),
        res.get("high"),
        res.get("low"),
        res.get("close") if res.get("close") is not None else res.get("last"),
        int(res.get("totalVolume")) if res.get("totalVolume") is not None else 0
    ))
    print("✅ Inserted AOT data ")
except Exception as e:
    print(f"❌ Error inserting AOT data: {e}")


✅ Inserted AOT data 


In [ ]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name} LIMIT 10")
df = pd.DataFrame(rows)
df.head(10)

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,PPPM,2025-06-23,NaN,NaN,NaN,NaN,0
2,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
3,TPCH,2025-06-23,NaN,NaN,NaN,NaN,0
4,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0
5,KPNREIT,2025-06-23,NaN,NaN,NaN,NaN,0
6,POLY,2025-05-16,6.70,6.70,6.70,6.70,1100
7,POLY,2025-06-23,NaN,NaN,NaN,NaN,0
8,QHBREIT,2025-05-16,3.62,3.62,3.58,3.62,240700
9,QHBREIT,2025-06-23,NaN,NaN,NaN,NaN,0


In [ ]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(2026, 7)

In [ ]:
df.head()

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,PPPM,2025-06-23,NaN,NaN,NaN,NaN,0
2,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
3,TPCH,2025-06-23,NaN,NaN,NaN,NaN,0
4,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0


In [ ]:
from datetime import datetime
import pandas as pd
import time

# โหลดรายชื่อหุ้น
try:
    symbols_df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/data_collection/settrade1.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ Excel — ใช้รายชื่อหุ้นสำรอง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ดึงข้อมูลหุ้น
def fetch_stock_data(symbol, market_data):
    try:
        res = market_data.get_quote_symbol(symbol)
        if not isinstance(res, dict):
            print(f"⚠️ ไม่พบข้อมูลสำหรับ {symbol}")
            return None

        return {
            "symbol": res.get("symbol"),
            "date": datetime.now().date(),
            "open": res.get("open") or res.get("last"),
            "high": res.get("high"),
            "low": res.get("low"),
            "close": res.get("close") or res.get("last"),
            "volume": int(res.get("totalVolume") or 0)
        }
    except Exception as e:
        print(f"❌ Error fetching {symbol}: {e}")
        return None

# --- แปลงข้อมูลเป็น DataFrame ---
all_data = []

for symbol in symbols:
    print(f"📦 กำลังโหลด {symbol} ...")
    stock_data = fetch_stock_data(symbol, market_data)
    if stock_data:
        all_data.append(stock_data)
    time.sleep(2)

df = pd.DataFrame(all_data)

📦 กำลังโหลด 24CS ...
📦 กำลังโหลด 2S ...
📦 กำลังโหลด 3BBIF ...
📦 กำลังโหลด A ...
📦 กำลังโหลด A5 ...
📦 กำลังโหลด AAI ...
📦 กำลังโหลด AAV ...
📦 กำลังโหลด ABM ...
📦 กำลังโหลด ACAP ...
📦 กำลังโหลด ACC ...
📦 กำลังโหลด ACE ...
📦 กำลังโหลด ACG ...
📦 กำลังโหลด ADB ...
📦 กำลังโหลด ADD ...
📦 กำลังโหลด ADVANC ...
📦 กำลังโหลด ADVICE ...
📦 กำลังโหลด AE ...
📦 กำลังโหลด AEONTS ...
📦 กำลังโหลด AF ...
📦 กำลังโหลด AFC ...
📦 กำลังโหลด AGE ...


KeyboardInterrupt: 

In [ ]:
print(df.head())

    symbol        date  close  high   low  open  volume
0     PPPM  2025-05-16   0.41  0.43  0.38  0.41  780900
1     PPPM  2025-06-23    NaN   NaN   NaN   NaN       0
2     TPCH  2025-05-16   2.78  2.80  2.72  2.78  258400
3     TPCH  2025-06-23    NaN   NaN   NaN   NaN       0
4  KPNREIT  2025-05-16    NaN   NaN   NaN   NaN       0


In [ ]:
def insert_bulk_data(df, session):
    insert_query = session.prepare("""
        INSERT INTO stock_price (symbol, date, open, high, low, close, volume)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """)
    for _, row in df.iterrows():
        session.execute(insert_query, (
            row['symbol'],
            row['date'],
            row['open'],
            row['high'],
            row['low'],
            row['close'],
            row['volume']
        ))
    print("✅ Insert ข้อมูลทั้งหมดเรียบร้อยแล้ว")

insert_bulk_data(df, session)


✅ Insert ข้อมูลทั้งหมดเรียบร้อยแล้ว


In [ ]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.head()

,symbol,date,close,high,low,open,volume
0,PPPM,2025-05-16,0.41,0.43,0.38,0.41,780900
1,PPPM,2025-06-23,NaN,NaN,NaN,NaN,0
2,TPCH,2025-05-16,2.78,2.80,2.72,2.78,258400
3,TPCH,2025-06-23,NaN,NaN,NaN,NaN,0
4,KPNREIT,2025-05-16,NaN,NaN,NaN,NaN,0


In [ ]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(2026, 7)